# Company Web Scraping through Google

Import all necessary packages for web scraping

In [1]:
import time
import csv
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Graduate\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Convert .txt file into .csv

In [ ]:
with open(r'C:\Users\Graduate\Desktop\Grad MSBA\Advanced Python\Final Project\company_info_final.txt', 'r') as infile, \
        open('company_info_final.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)

    # Write the header row
    writer.writerow(['Company', 'NAICS on SoS site'])

    # Loop through the lines in the input file
    fields = {}
    for line in infile:
        # Strip whitespace from the line
        line = line.strip()

        # If the line starts with "Company:", start a new record
        if line.startswith('Company:'):
            # Add the previous record to the output file, if any
            if fields:
                # Remove non-numeric characters from NAICS code
                naics = fields.get('NAICS on SoS site', '')
                naics = re.sub(r'\D', '', naics)
                writer.writerow([fields.get('Company', ''), naics])

            # Start a new dictionary of fields
            fields = {}

        # Split the line into fields based on the delimiter
        if ':' in line:
            key, value = [s.strip() for s in line.split(':', 1)]
            if key == 'NAICS on SoS site':
                fields[key] = value.strip()

            else:
                fields[key] = value

    # Write the last record to the output file
    if fields:
        # Remove non-numeric characters from NAICS code
        naics = fields.get('NAICS on SoS site', '')
        naics = re.sub(r'\D', '', naics)
        writer.writerow([fields.get('Company', ''), naics])

print('CSV file generated successfully!')

Conduct Web Scraping of Company Names

In [ ]:
# Load the DataFrame with company and description columns
df = pd.read_csv("C:/Users/Graduate/Desktop/Grad MSBA/Advanced Python/Final Project/company_info_final.csv")

# Load the NAICS_subcategories file
naics_subcategories = pd.read_csv("NAICS_subcategories.csv")

# Merge the two DataFrames on the first 3 digits of the NAICS code
df = pd.merge(df, naics_subcategories, left_on=df["NAICS on SoS site"].astype(str).str[:3], right_on=naics_subcategories["NAICS Code"].astype(str).str[:3], how="left")

# Define an empty dictionary to store the results
keyword_dict = {}

# Define a set of stopwords
stop_words = set(stopwords.words('english'))

# Add keyword_count and found_count columns to the DataFrame
df["keyword_count"] = 0
df["found_count"] = 0

# Start the web driver
driver = webdriver.Chrome()

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    company = row["Company"].strip()
    description = row["Description"]
        
    # Navigate to Google.com
    driver.get("https://www.google.com/")
        
    # Find the search box and enter the company name
    search_box = driver.find_element('name', 'q')
    search_box.send_keys(company + " " + 'manufacturing')
    search_box.submit()
        
    # Wait for the search results to load
    driver.implicitly_wait(10)
        
    # Find the first search result link that is not an advertisement
    search_results = driver.find_elements('css selector', "div.tF2Cxc")
    for result in search_results:
        try:
            search_link = result.find_element('tag name', 'a')
            if 'http' in search_link.get_attribute('href') and 'google' not in search_link.get_attribute('href'):
                search_link.click()
                break
        except:
            pass
        
    # Wait for the resulting page to load
    driver.implicitly_wait(10)
        
    # Loop through each keyword for the current company
    for word in (description).split():
        if word.lower() not in stop_words:
            if word.lower() in description.lower():
                keyword_dict.setdefault(word.lower(), []).append(1)
                # Increment the found count for the current row
                df.at[index, "found_count"] += 1
            else:
                keyword_dict.setdefault(word.lower(), []).append(0)
            # Increment the keyword count for the current row
            df.at[index, "keyword_count"] += 1 
        
    # Add the found count to the DataFrame
    df.at[index, "found_count"] = df.at[index, "found_count"]
        
    # Calculate the percentage of keywords found and add to the DataFrame
    percentage = df.at[index, "found_count"] / df.at[index, "keyword_count"] if df.at[index, "keyword_count"] > 0 else 0
    df.at[index, "found_percentage"] = percentage
        
# Close the web driver
driver.quit()

!TEST OF CODE FROM ABOVE!

In [13]:
# Load the DataFrame with company and description columns
df = pd.read_csv("C:/Users/Graduate/Desktop/Grad MSBA/Advanced Python/Final Project/company_info_final.csv")

# Load the NAICS_subcategories file
naics_subcategories = pd.read_csv("NAICS_subcategories.csv")

# Merge the two DataFrames on the first 3 digits of the NAICS code
df = pd.merge(df, naics_subcategories, left_on=df["NAICS on SoS site"].astype(str).str[:3], right_on=naics_subcategories["NAICS Code"].astype(str).str[:3], how="left")

# Define an empty dictionary to store the results
keyword_dict = {}

# Define a set of stopwords
stop_words = set(stopwords.words('english'))

# Add keyword_count and found_count columns to the DataFrame
df["keyword_count"] = 0
df["found_count"] = 0

# Loop through the first 10 rows in the DataFrame
for index in range(5):
    row = df.loc[index]
    company = row["Company"].strip()
    description = row["Description"]
        
    # Start the web driver
    driver = webdriver.Chrome()
        
    # Navigate to Google.com
    driver.get("https://www.google.com/")
        
    # Find the search box and enter the company name
    search_box = driver.find_element('name', 'q')
    search_box.send_keys(company + " " + 'manufacturing')
    search_box.submit()
        
    # Wait for the search results to load
    driver.implicitly_wait(10)
        
    # Find the first search result link that is not an advertisement
    search_results = driver.find_elements('css selector', "div.tF2Cxc")
    for result in search_results:
        try:
            search_link = result.find_element('tag name', 'a')
            if 'http' in search_link.get_attribute('href') and 'google' not in search_link.get_attribute('href'):
                search_link.click()
                break
        except:
            pass
        
    # Wait for the resulting page to load
    driver.implicitly_wait(10)
        
    # Loop through each keyword for the current company
    for word in (description).split():
        if word.lower() not in stop_words:
            if word.lower() in description.lower():
                keyword_dict.setdefault(word.lower(), []).append(1)
                # Increment the found count for the current row
                df.at[index, "found_count"] += 1
            else:
                keyword_dict.setdefault(word.lower(), []).append(0)
            # Increment the keyword count for the current row
            df.at[index, "keyword_count"] += 1 
        
    # Add the found count to the DataFrame
    df.at[index, "found_count"] = df.at[index, "found_count"]
        
    # Calculate the percentage of keywords found and add to the DataFrame
    percentage = df.at[index, "found_count"] / df.at[index, "keyword_count"] if df.at[index, "keyword_count"] > 0 else 0
    df.at[index, "found_percentage"] = percentage
        
    # Close the current tab
    driver.close()
    
# Close the web driver
driver.quit()

In [14]:
# Save the updated DataFrame to a new Excel file
df.to_excel("Company_Web_Scraped.xlsx", index=False)